# 分词

中文分词技术指的是将中文文本按照一定的规则切分成一个一个的词语的方法和技术。由于中文没有像英文那样的明确的词与词之间的分隔符，因此中文分词技术在中文信息处理领域中非常重要，是自然语言处理的基础。

> 中文分词技术可以分为基于词典的分词和基于统计的分词两种方法。

1. 基于词典的分词方法是将文本中的每个字符与一个词典中的词进行匹配，匹配成功的字符组成一个词语。这种方法的优点是速度快、准确率高，但缺点是无法处理未登记在词典中的新词。
2. 基于统计的分词方法则是通过对大量文本进行分析，建立语言模型，利用统计学方法预测每个字符是否可以组成一个词语。这种方法的优点是可以处理未登记在词典中的新词，但缺点是需要大量的语料库训练和计算，而且准确率可能受到语料库的质量和数量的影响。

中文分词技术在搜索引擎、自然语言处理、机器翻译、信息检索等领域都有广泛的应用，是中文信息处理中不可或缺的基础技术。

# IK

## Install

1. 先在改地址下载`IK`，注意`ElasticSearch`和`IK`插件版本的对应。: [elasticsearch-analysis-ik](https://github.com/medcl/elasticsearch-analysis-ik)
2. 将文件解压到，`ik`文件夹需要自己创建：
    ```path
    \elasticsearch-8.7.0\plugins\ik
    ```
3. 安装配置好之后需要重启`ES`服务。


> 其他参考：ES[分析插件](https://www.elastic.co/guide/en/elasticsearch/plugins/current/analysis.html)

> Appendix

1. 设置所有索引的默认分词器为`ik`分词：在`ElasticSearch`的配置文件`config/elasticsearch.yml`中的最后一行添加参数`index.analysis.analyzer.default.type: ik`。
2. 也可以通过设置`mapping`来使用`ik`分词。

## IK分词介绍

`IK`分词器的两种分词模式：

1. `ik_max_word`: 会将文本做最细粒度的拆分，比如会将`中华人民共和国国歌`拆分为`中华人民共和国 / 中华人民 / 中华 / 华人 / 人民共和国 / 人民 / 人 / 民 / 共和国 / 共和 / 和 / 国国 / 国歌`，会穷尽各种可能的组合；
2. `ik_smart`: 会做最粗粒度的拆分，比如会将`中华人民共和国国歌`拆分为`中华人民共和国 / 国歌`。

## 验证安装

In [2]:
import requests

def es_ik_words(text, url=None, analyzer='ik_max_word'):
    if not url:
        url = "http://localhost:9200/_analyze"
    data = {"analyzer": analyzer, "text": text}
    response = requests.post(url, json=data)
    tokens = [token["token"] for token in response.json()["tokens"]]
    return tokens

### ES默认的分词方式

#### standard

> `standard` 是默认的分析器。它提供了基本语法的标记化（基于 Unicode 文本分割算法），适用于大多数语言。分词方式：区分中英文，英文按照空格切分同时大写转小写；中文按照单个词分词。

In [3]:
es_ik_words(text='你好，我来自浙江省杭州市西湖区。', analyzer='standard')

['你', '好', '我', '来', '自', '浙', '江', '省', '杭', '州', '市', '西', '湖', '区']

In [4]:
es_ik_words(text='This is a test.', analyzer='standard')

['this', 'is', 'a', 'test']

#### Simple Analyzer

> simple 分析器当它遇到只要不是字母的字符，就将文本解析成term，而且所有的 term 都是小写的。
【分词方式】 先按照空格分词，英文大写转小写，不是英文不分词。

In [5]:
es_ik_words(text='你好，我来自浙江省杭州市西湖区。', analyzer='simple')

['你好', '我来自浙江省杭州市西湖区']

In [6]:
es_ik_words(text='This is a test.', analyzer='simple')

['this', 'is', 'a', 'test']

#### Whitespace Analyzer

>【分词方式】 按空格分词，英文不区分大小写，中文不再分词

In [7]:
es_ik_words(text='你好，我来自浙江省杭州市西湖区。', analyzer='whitespace')

['你好，我来自浙江省杭州市西湖区。']

In [8]:
es_ik_words(text='This is a test.', analyzer='whitespace')

['This', 'is', 'a', 'test.']

### IK

#### ik_max_word

In [9]:
es_ik_words(text='你好，我来自浙江省杭州市西湖区。', analyzer='ik_max_word')

['你好', '我', '来自', '浙江省', '浙江', '省', '杭州市', '杭州', '市', '西湖区', '西湖', '湖区']

#### ik_smart

In [10]:
es_ik_words(text='你好，我来自浙江省杭州市西湖区。', analyzer='ik_smart')

['你好', '我', '来自', '浙江省', '杭州市', '西湖区']

In [11]:
text = '''
由清朝作家曹雪芹创作，被誉为中国古代小说的巅峰之作。
小说描绘了贾宝玉、林黛玉等人的生活，以及他们所处的大观园，
通过描绘人物的情感和性格，反映了封建社会的种种矛盾和不幸。
小说以其深刻的思想性、细腻的描写和独特的艺术风格而闻名于世。
'''

' / '.join(es_ik_words(text=text, analyzer='ik_smart'))

'由 / 清朝 / 作家 / 曹雪芹 / 创作 / 被誉为 / 中国古代 / 小说 / 的 / 巅峰 / 之作 / 小说 / 描绘 / 了 / 贾宝玉 / 林黛玉 / 等人 / 的 / 生活 / 以及 / 他们 / 所处 / 的 / 大观园 / 通过 / 描绘 / 人物 / 的 / 情感 / 和 / 性格 / 反映 / 了 / 封建社会 / 的 / 种种 / 矛盾 / 和 / 不幸 / 小说 / 以其 / 深刻 / 的 / 思想性 / 细腻 / 的 / 描写 / 和 / 独特 / 的 / 艺术风格 / 而 / 闻名于世'

## 分词效果对比

### 默认文档创建Index并导入数据

In [12]:
import pandas as pd
from elasticsearch_dsl import (
    connections, tokenizer, analyzer,
    Index, Document, Text, Integer, Boolean, Keyword)

# 创建 Elasticsearch 连接
connections.create_connection(hosts=["localhost"])

<Elasticsearch([{'host': 'localhost'}])>

In [13]:
# 指定要创建的索引名称
index_name = "books_normal"

# 如果存在则删除已经创建的索引
if Index(index_name).exists():
    Index(index_name).delete()

# 定义索引和映射
class Books(Document):
    title = Text()
    description = Text()
    category = Keyword()
    price = Integer()
    in_stock = Boolean()

    class Index:
        name = index_name
        
# 创建索引并将映射与索引关联
Books.init()

In [14]:
# 导入数据
df_books = pd.read_csv('./data/books.csv', encoding='gbk')
for row_no, row in df_books.iterrows():
    book = Books(
        title=row.title, description=row.description, 
        category=row.category, price=row.price, in_stock=row.in_stock)
    book.save()

In [15]:
ik_index = Index(index_name)
# get_mapping
ik_index.get_mapping()

{'books_normal': {'mappings': {'properties': {'category': {'type': 'keyword'},
    'description': {'type': 'text'},
    'in_stock': {'type': 'boolean'},
    'price': {'type': 'integer'},
    'title': {'type': 'text'}}}}}

In [16]:
# get_mapping
ik_index.get_settings()

{'books_normal': {'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}},
    'number_of_shards': '1',
    'provided_name': 'books_normal',
    'creation_date': '1688291697496',
    'number_of_replicas': '1',
    'uuid': 'xn5B80lZQreO_As_i77oDQ',
    'version': {'created': '8070099'}}}}}

### IK分词创建Index并导入数据

In [17]:
import pandas as pd
from elasticsearch_dsl import (
    connections, tokenizer, analyzer,
    Index, Document, Text, Integer, Boolean, Keyword)

# 创建 Elasticsearch 连接
connections.create_connection(hosts=["localhost"])

<Elasticsearch([{'host': 'localhost'}])>

In [18]:
# 指定要创建的索引名称
index_name = "books_ik"

# 定义分析器和分词器
ik_tokenizer = tokenizer('ik_smart')
ik_analyzer = analyzer('ik_smart', tokenizer=ik_tokenizer)

# 如果存在则删除已经创建的索引
if Index(index_name).exists():
    Index(index_name).delete()

# 定义索引和映射
class Books(Document):
    title = Text(analyzer='ik_smart', search_analyzer=ik_analyzer)
    description = Text(analyzer='ik_smart', search_analyzer=ik_analyzer)
    category = Keyword()
    price = Integer()
    in_stock = Boolean()

    class Index:
        name = index_name
        settings = {
            "analysis": {
                "analyzer": {
                    "ik_smart": {
                        "tokenizer": "ik_smart"
                    }
                },
                "search_analyzer": {
                    "ik_smart": {
                        "tokenizer": "ik_smart"
                    }
                },
            }
        }
        
# 创建索引并将映射与索引关联
Books.init()

In [19]:
# 导入数据
df_books = pd.read_csv('./data/books.csv', encoding='gbk')
for row_no, row in df_books.iterrows():
    book = Books(
        title=row.title, description=row.description, 
        category=row.category, price=row.price, in_stock=row.in_stock)
    book.save()

In [20]:
ik_index = Index(index_name)
# get_mapping
ik_index.get_mapping()

{'books_ik': {'mappings': {'properties': {'category': {'type': 'keyword'},
    'description': {'type': 'text', 'analyzer': 'ik_smart'},
    'in_stock': {'type': 'boolean'},
    'price': {'type': 'integer'},
    'title': {'type': 'text', 'analyzer': 'ik_smart'}}}}}

In [21]:
# get_mapping
ik_index.get_settings()

{'books_ik': {'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}},
    'number_of_shards': '1',
    'provided_name': 'books_ik',
    'creation_date': '1688291789433',
    'analysis': {'analyzer': {'ik_smart': {'type': 'custom',
       'tokenizer': 'ik_smart'}}},
    'number_of_replicas': '1',
    'uuid': '96YZrZysQo-KLoFr8R0Jjw',
    'version': {'created': '8070099'}}}}}

### `IK`分词查询与默认分词查询对比

In [23]:
from dsl_utils import *
from elasticsearch_dsl import Search
from elasticsearch_dsl.query import MatchPhrase

# 创建一个查询对象
search = Search(index='books_ik')
# 创建一个 MatchPhrase 查询
query = MatchPhrase(description='曹芹')
search_query(search, query, columns=['title', 'description'])

In [24]:
# 创建一个 MatchPhrase 查询
query = MatchPhrase(description='曹雪')
search_query(search, query, columns=['title', 'description'])

In [25]:
# 创建一个 MatchPhrase 查询
query = MatchPhrase(description='曹雪芹')
search_query(search, query, columns=['title', 'description'])

title: 红楼梦
description: 由清朝作家曹雪芹创作，被誉为中国古代小说的巅峰之作。小说描绘了贾宝玉、林黛玉等人的生活，以及他们所处的大观园，通过描绘人物的情感和性格，反映了封建社会的种种矛盾和不幸。小说以其深刻的思想性、细腻的描写和独特的艺术风格而闻名于世。


**以下为默认效果**

In [27]:
# 创建一个查询对象
search = Search(index='books_normal')
# 创建一个 MatchPhrase 查询
query = MatchPhrase(description='曹芹')
search_query(search, query, columns=['title', 'description'])

In [28]:
# 创建一个 MatchPhrase 查询
query = MatchPhrase(description='曹雪')
search_query(search, query, columns=['title', 'description'])

title: 红楼梦
description: 由清朝作家曹雪芹创作，被誉为中国古代小说的巅峰之作。小说描绘了贾宝玉、林黛玉等人的生活，以及他们所处的大观园，通过描绘人物的情感和性格，反映了封建社会的种种矛盾和不幸。小说以其深刻的思想性、细腻的描写和独特的艺术风格而闻名于世。


In [29]:
# 创建一个 MatchPhrase 查询
query = MatchPhrase(description='曹雪芹')
search_query(search, query, columns=['title', 'description'])

title: 红楼梦
description: 由清朝作家曹雪芹创作，被誉为中国古代小说的巅峰之作。小说描绘了贾宝玉、林黛玉等人的生活，以及他们所处的大观园，通过描绘人物的情感和性格，反映了封建社会的种种矛盾和不幸。小说以其深刻的思想性、细腻的描写和独特的艺术风格而闻名于世。


# jieba分词

*目前，`jieba`分词最高支持到`ES`版本为`7.7`，这里就不展示了。*

# 参考

- [medcl/elasticsearch-analysis-ik](https://github.com/medcl/elasticsearch-analysis-ik)
- [elasticsearch怎么查看使用的分词器？](https://www.zhihu.com/question/512654810)
- [Elasticsearch7.X集成jieba分词插件](https://blog.csdn.net/weixin_42326851/article/details/124615116)
- [ElasticSearch-IK分词使用踩坑总结](https://www.jianshu.com/p/f178e59ffaf2)
- [ES中文分词-IK分词](https://zhuanlan.zhihu.com/p/357625669)


-----